In [1]:
import random
import numpy as np
from tqdm import tnrange, tqdm_notebook
import tensorflow as tf


def score(context, response):
    return random.random()


def best_response(context, candidates):
    index = np.argmax([score(context, response) for response in candidates])
    return candidates[index]


def parse_dialogs(filename):
    dialogs = []
    with open(filename, 'r') as f:
        dialog = []
        for line in f:
            if line.strip() == '':
                dialogs.append(dialog)
                dialog = []
            else:
                user_utt, bot_utt = line.strip().split('\t')
                utt_num = user_utt.split(' ')[0]
                user_utt = ' '.join(user_utt.split(' ')[1:])
                dialog.append((utt_num, user_utt, bot_utt))
    return dialogs


def parse_candidates(filename):
    with open(filename, 'r') as f:
        return [' '.join(line.strip().split(' ')[1:]) for line in f]            

    
def responses_accuracy(dialogs, candidates):
    correct = 0
    count = 0
    for dialog in dialogs:
        for _, user_utt, bot_utt in dialog:
            count += 1
            context = user_utt
            response = best_response(context, candidates)
            if response == bot_utt:
                correct += 1
    return correct / count, correct, count


def build_vocab_to_ind_map(dialogs):
    vocab = set()
    for d in dialogs:
        for _, user_utt, bot_utt in d:
            vocab = vocab.union(user_utt.split(' ') + bot_utt.split(' '))
    vocab = sorted(list(vocab))
    
    cntr = 0
    vocab_ind_map = {}
    for w in vocab:
        vocab_ind_map[w] = cntr
        cntr += 1
    return vocab_ind_map


def build_vec(vocab_ind_map, utt):
    vocab_len = len(vocab_ind_map.keys())
    vec = np.zeros((vocab_len, 1))
    for w in utt.split(' '):
        vec[vocab_ind_map[w]] += 1
    return vec


def get_vec_set(dialogs, vocab_ind_map):
    vec_set = []
    for d in dialogs:
        for _, user_utt, bot_utt in d:
            x = build_vec(vocab_ind_map, user_utt)
            y = build_vec(vocab_ind_map, bot_utt)
            vec_set.append([x, y])
    return vec_set

In [2]:
train_set_task1_dialogs = parse_dialogs('dataset/dialog-bAbI-tasks/dialog-babi-task1-API-calls-trn.txt')
dev_set_task1_dialogs = parse_dialogs('dataset/dialog-bAbI-tasks/dialog-babi-task1-API-calls-dev.txt')
candidates = parse_candidates('dataset/dialog-bAbI-tasks/dialog-babi-candidates.txt')
vocab_to_ind_map = build_vocab_to_ind_map(train_set_task1_dialogs)
vec_set = get_vec_set(train_set_task1_dialogs, vocab_to_ind_map)

In [39]:
D = 20
V = len(vocab_to_ind_map)

context = tf.placeholder(dtype=tf.float32, name='Context', shape=[V, 1])
neg_context = tf.placeholder(dtype=tf.float32, name='NegativeContext', shape=[V, 1])
response = tf.placeholder(dtype=tf.float32, name='Response', shape=[V, 1])
A_var = tf.Variable(tf.random_uniform([D, V], -1.0, 1.0))
B_var = tf.Variable(tf.random_uniform([D, V], -1.0, 1.0))

f_pos = tf.matmul(tf.transpose(tf.matmul(A_var, context)), tf.matmul(B_var, response))
f_neg = tf.matmul(tf.transpose(tf.matmul(A_var, neg_context)), tf.matmul(B_var, response))

m = 0.01
loss = tf.nn.relu(f_pos - f_neg + m)

LR = 0.001
optimizer = tf.train.GradientDescentOptimizer(LR).minimize(loss)

In [48]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    avg_loss = 0.0
    for _ in tnrange(10):
        for x, y in tqdm_notebook(vec_set):
            y_negs = random.sample(vec_set, 1)
            for _, y_neg in y_negs:
                loss_elem = sess.run([loss, optimizer], feed_dict={context: x, response: y, neg_context: y_neg})
                avg_loss += loss_elem[0][0]
        avg_loss = avg_loss / len(vec_set)
        print(loss_elem, avg_loss)
        val_pos = sess.run([f_pos], feed_dict={context: vec_set[-1][0], response: vec_set[-1][1]})
        val_neg = sess.run([f_neg], feed_dict={neg_context: vec_set[1][0], response: vec_set[-1][1]})
        print(val_pos, val_neg)


[array([[ 0.]], dtype=float32), None] [ 1.06638396]
[array([[-6.5842762]], dtype=float32)] [array([[-3.84863114]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.08665579]
[array([[-8.40829372]], dtype=float32)] [array([[-1.92801046]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.01955559]
[array([[-8.96854973]], dtype=float32)] [array([[-1.16675735]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.01130519]
[array([[-9.40524387]], dtype=float32)] [array([[-0.67194474]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.0103422]
[array([[-9.55266953]], dtype=float32)] [array([[-0.52301461]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.00429054]
[array([[-9.6255188]], dtype=float32)] [array([[-0.49306503]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.00364747]
[array([[-9.76230431]], dtype=float32)] [array([[-0.37666884]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.00326872]
[array([[-9.89340878]], dtype=float32)] [array([[-0.23280609]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.0029568]
[array([[-9.98664951]], dtype=float32)] [array([[-0.16421798]], dtype=float32)]


[array([[ 0.]], dtype=float32), None] [ 0.00120396]
[array([[-9.99772358]], dtype=float32)] [array([[-0.1633212]], dtype=float32)]

